In [1]:
import multiprocessing
print(multiprocessing.cpu_count())

import psutil
print(f"Available memory before training: {psutil.virtual_memory().available / 1e9:.2f} GB")

10
Available memory before training: 6.10 GB


# Diabetes Readmission – XGBoost Gradient Boosting

## Introduction

This notebook implements XGBoost (eXtreme Gradient Boosting) for predicting hospital readmission within 30 days for diabetic patients. We use the preprocessed dataset designed for tree-based methods, which includes:

- Full dataset: All encounters retained (101,763 records), as tree-based methods can handle correlated observations
- Binary and count features: ICD-9 diagnostic codes expanded into both indicator variables and count-based features
- Categorical encoding: Ordinal encoding for categorical variables (optimal for tree-based models)
- Raw numeric features: No scaling required as XGBoost is invariant to monotonic transformations

## Methodology

**No Class Imbalance Handling**: XGBoost naturally handles class imbalance through built-in mechanisms like `scale_pos_weight` parameter, eliminating the need for synthetic sampling techniques.

**Gradient Boosting Approach**: XGBoost builds an ensemble of weak decision trees sequentially, where each tree corrects the errors of previous trees. Key advantages include:
- Feature interaction detection: Automatically captures complex non-linear relationships
- Missing value handling: Native support for missing data without imputation
- Regularization: Built-in L1 and L2 regularization prevents overfitting

**Hyperparameter Optimization**: Using Optuna's intelligent search across 9 key parameters:
- Tree structure: `n_estimators`, `max_depth`, `min_child_weight`
- Learning dynamics: `learning_rate`, `subsample`, `colsample_bytree`
- Regularization: `reg_alpha`, `reg_lambda`
- Class handling: `scale_pos_weight`

**Preprocessing Pipeline**: Ordinal encoding for categorical features while preserving numeric features as-is, resulting in ~147 features optimized for tree-based learning.

The goal is to leverage XGBoost's ability to capture complex feature interactions and non-linear patterns that linear models cannot detect, while maintaining robust performance through proper regularization.

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pickle
import time

In [3]:
token = 'f11' # iteratable by the user as we try new things
randy = 42 # random value insertion for repeatability
random_forests = pd.read_pickle("../models/randomForests.pkl") # See prior notebook, p02.

In [4]:
random_forests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101763 entries, 0 to 101765
Columns: 147 entries, encounter_id to count_E990_E999
dtypes: bool(4), float64(6), int64(115), object(22)
memory usage: 112.2+ MB


## Memory Optimization

The `optimize_dtypes()` function reduces memory usage by downcasting numeric types to their smallest sufficient representation:
- `int64` → `int8/int16/int32` based on value ranges
- `float64` → `float32` when precision allows

This optimization is particularly valuable for large datasets and memory-intensive operations like SMOTE resampling.

In [5]:
def optimize_dtypes(df):
    
    """
    Here we convert some of our columns intelligently to save on memory & time
    """
    
    for col in df.columns:
        col_type = df[col].dtype

        if col_type == 'int64':
            c_min = df[col].min()
            c_max = df[col].max()

            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)

        elif col_type == 'float64':
            c_min = df[col].min()
            c_max = df[col].max()

            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)

    return df

In [6]:
random_forests = optimize_dtypes(random_forests)

In [7]:
random_forests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101763 entries, 0 to 101765
Columns: 147 entries, encounter_id to count_E990_E999
dtypes: bool(4), float32(6), int16(1), int32(2), int8(112), object(22)
memory usage: 32.4+ MB


In [8]:
import xgboost as xgb
from xgboost import XGBClassifier

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score

/Users/cwaters/diabetes/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model Evaluation and Persistence Function

The `evaluate_and_save_pipeline()` function provides standardized evaluation across all modeling approaches in this project:

**Comprehensive Metrics Calculation:**
- **Classification performance**: Accuracy, precision, recall, F1-score, specificity
- **Probability-based metrics**: ROC curve data and AUC score for threshold optimization
- **Confusion matrix**: True/false positive/negative counts for detailed performance analysis
- **Prediction arrays**: Both binary predictions and probability scores for ensemble building

**Standardized Output Format:**
All metrics are saved in identical pickle format enabling:
- Direct performance comparison across different model types
- Consistent evaluation methodology regardless of underlying algorithm
- Easy integration into ensemble methods and model selection workflows
- Reproducible results with preserved prediction arrays

**Model Persistence:**
Trained pipelines are saved with preprocessing steps intact, ensuring deployment-ready models that can handle new data with the same
transformations applied during training.

This standardization is critical for fair model comparison and supports the ensemble modeling approach in later notebooks.

In [9]:
def evaluate_and_save_pipeline(pipeline, namestring, token, 
                                X_train, X_test, 
                                y_train, y_test,
                                console_out = False):
    """
    Evaluates a trained pipeline and saves metrics to a pickle file.
    """

    # Input validation
    if any(v is None for v in [X_train, X_test, y_train, y_test]):
        raise ValueError("X_train, X_test, y_train, or y_test must not be None.")

    # Convert to numpy if needed
    y_train = y_train.values if hasattr(y_train, "values") else y_train
    y_test = y_test.values if hasattr(y_test, "values") else y_test

    # Make predictions once
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)

    # Get probability predictions
    if hasattr(pipeline, "predict_proba"):
        y_test_pred_pct = pipeline.predict_proba(X_test)[:, 1]
    elif hasattr(pipeline, "decision_function"):
        y_test_pred_pct = pipeline.decision_function(X_test)
    else:
        raise AttributeError("Pipeline needs predict_proba() or decision_function() for ROC/AUC.")

    # Classification metrics (not regression metrics)
    accuracy = pipeline.score(X_test, y_test)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)  # Same as sensitivity
    f1 = f1_score(y_test, y_test_pred)

    # Confusion matrix metrics
    tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_pct)
    roc_auc = auc(fpr, tpr)

    # Safe access to classes
    classes_ = getattr(pipeline, 'classes_', np.unique(y_train))

    # Save metrics
    pickle_metrics = {
        'model_version': f"{token}_{namestring}",
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'specificity': specificity,
        'roc_auc': roc_auc,
        'y_test': y_test,
        'y_train_pred': y_train_pred,
        'y_test_pred': y_test_pred,
        'y_test_pred_proba': y_test_pred_pct,
        'display_labels': classes_,
        'confusion_matrix': {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp},
        'roc_curve': {'fpr': fpr, 'tpr': tpr, 'thresholds': thresholds},

        # SHAP-specific additions
        'shap_data': {
            'model': pipeline,
            'X_train_processed': pipeline.named_steps['preprocessor'].transform(X_train),
            'X_test_processed': pipeline.named_steps['preprocessor'].transform(X_test),
            'feature_names': pipeline.named_steps['preprocessor'].get_feature_names_out(),
            'original_feature_names': list(X_train.columns)
        }
    }

    # Save to file
    filename = f"../models/fits_pickle_{token}_{namestring}.pkl"
    with open(filename, "wb") as file:
        pickle.dump(pickle_metrics, file)

    if console_out:
            # Print summary
            print(f"Metrics saved to {filename}")
            print(f'Accuracy:    {accuracy:.4f}')
            print(f'Precision:   {precision:.4f}')
            print(f'Recall:      {recall:.4f}')
            print(f'F1-Score:    {f1:.4f}')
            print(f'Specificity: {specificity:.4f}')
            print(f'ROC AUC:     {roc_auc:.4f}')

            # Plot confusion matrix
            cm = confusion_matrix(y_test, y_test_pred)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes_)
            disp.plot(cmap=plt.cm.Blues)
            plt.title(f"Confusion Matrix - {namestring}")
            plt.show()
    
    return pickle_metrics

In [10]:
X = random_forests.drop(["readmitted"], axis=1)
y = random_forests["readmitted"]

## Feature Type Usage

**exclude_features**: Used as a filter when defining the other feature types - ensures ID columns and target variable don't get included in modeling features.

**numeric_features**:
- Fed into "passthrough" in the ColumnTransformer preprocessor
- Preserves original values without scaling (XGBoost handles different scales naturally)
- These remain as continuous variables for tree splitting decisions

**boolean_features**: 
- Combined with object_features and passed to OrdinalEncoder
- Converted to integers (0, 1) rather than one-hot encoded
- More efficient representation for tree-based models

**object_features**: 
- Combined with boolean_features and passed to OrdinalEncoder  
- Each category mapped to integer codes (0, 1, 2, ...)
- `handle_unknown="use_encoded_value", unknown_value=-1` handles unseen categories

**Combined usage**:
- ColumnTransformer applies different preprocessing: passthrough to numeric, OrdinalEncoder to categorical
- No feature expansion - maintains original ~147 features (vs 2,871 with one-hot encoding)
- `categorical_features` list tracks column indices for XGBoost's native categorical handling

The separation optimizes preprocessing for tree-based learning - numeric features retain their distributions for optimal splits, while categorical features use compact ordinal encoding that XGBoost can efficiently partition.

In [11]:
# Training features to include
exclude_features = ["patient_nbr", "encounter_id", "readmitted"]
numeric_features = [
    col
    for col in X.columns
    if col not in exclude_features and pd.api.types.is_numeric_dtype(X[col])
]
boolean_features = [
    col for col in X.columns if col not in exclude_features and X[col].dtype == "bool"
]
object_features = [
    col for col in X.columns if col not in exclude_features and X[col].dtype == "object"
]
categorical_features = [
    X.columns.get_loc(col) for col in object_features + boolean_features
]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=randy, stratify=y
)

## Preprocessing with ColumnTransformer

This applies XGBoost-optimized preprocessing that differs significantly from linear models:

1. **"num" step**: Applies "passthrough" to numeric_features
   - No scaling applied - XGBoost is invariant to monotonic transformations
   - Preserves original value ranges and distributions
   
2. **"cat" step**: Applies OrdinalEncoder() to categorical features
   - Converts categories to integers rather than one-hot encoding
   - `handle_unknown="use_encoded_value", unknown_value=-1`: Maps unseen categories to -1
   - Much more efficient than one-hot encoding for tree-based models
   - Preserves ordinality where it exists naturally

Why This Approach for XGBoost:
- Tree-based models split on single feature values, making ordinal encoding optimal
- Avoids the curse of dimensionality from one-hot encoding (~147 features vs 2,871)
- XGBoost can learn optimal splits regardless of numeric scale
- Missing value encoding (-1) allows XGBoost's native missing value handling to work effectively

The result is a compact, efficient feature representation optimized for gradient boosting.

In [13]:
preprocessor_xgb = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),  # Keep as-is
        ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
        object_features + boolean_features)  # Encode multiple columns
    ]
)

# This is also an option
# preprocessor_xgb = ColumnTransformer(
#     transformers=[
#         ('num', MinMaxScaler(), numeric_features),
#         ('cat', OneHotEncoder(drop='first', sparse_output=True, handle_unknown='ignore'), object_features)
#     ])

Preprocessing is done inside the pipeline to ensure consistent transformation between training and testing.

In [14]:
# Final check of NaN's before training
print("Checking for NaNs in categorical columns...")
categorical_cols = object_features + boolean_features
nan_check = {}

for col in categorical_cols:
    nan_count = X[col].isna().sum()
    if nan_count > 0:
        nan_check[col] = nan_count

if nan_check:
    print("STOP! Still have NaNs:")
    for col, count in nan_check.items():
        print(f"  {col}: {count} NaNs")
    print("\nFill these before training!")
else:
    print("No NaNs found in categorical columns - safe to train!")

Checking for NaNs in categorical columns...
No NaNs found in categorical columns - safe to train!


## Hyperparameter Search Strategy

**Log-Scale Parameter Tuning:**
Several hyperparameters are sampled on logarithmic scales using `log=True`:

- `learning_rate` (0.001-1.0, log scale): Learning rates vary exponentially in effectiveness - the difference between 0.001 and 0.01 is as significant as between 0.1 and 1.0
- `reg_lambda` & `reg_alpha` (0.001-1000, log scale): Regularization strength spans several orders of magnitude - linear sampling would oversample high values and miss important low-regularization regions

**Why Log Scaling Matters:**
- Even exploration: Ensures equal search attention across exponential ranges (e.g., 0.001, 0.01, 0.1, 1.0)
- Efficient optimization: Optuna's TPE sampler works more effectively when parameter importance is evenly distributed
- Practical relevance: Most ML hyperparameters exhibit exponential rather than linear sensitivity

**Linear vs Categorical Parameters:**
- `subsample`, `colsample_bytree`: Linear scale (0.5-1.0) as effects are more uniform across this range
- `n_estimators`, `max_depth`: Integer ranges where each increment has roughly equal impact
- `scale_pos_weight`: Categorical values based on class ratio multiples

This mixed approach optimizes search efficiency while respecting the natural scaling properties of each hyperparameter.

In [15]:
def objective(trial):
    # Define hyperparameters to search
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0001, 100, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0001, 100, log=True),
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 1, 3),
        'random_state': randy,
        'verbosity': 0
    }

    # Preprocess the entire training set once
    X_train_proc = preprocessor_xgb.fit_transform(X_train)

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train_proc, label=y_train)

    # Use xgboost.cv with pruning callback
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=params['n_estimators'],
        nfold=5,
        stratified=True,
        shuffle=True,
        seed=randy,
        early_stopping_rounds=10,
        callbacks=[XGBoostPruningCallback(trial, 'test-auc')]
    )

    # Return the best CV score
    return cv_results['test-auc-mean'].iloc[-1]

In [16]:
# Callbacks for monitoring
def print_best_callback(study, trial):
    print(f"Trial {trial.number}: {trial.value:.4f} (best: {study.best_value:.4f})")

def save_best_callback(study, trial):
    if study.best_trial == trial:
        # Save best params so far
        with open(f"../models/{token}_XGB_best_params.pkl", "wb") as f:
            pickle.dump(study.best_params, f)

In [17]:
%%time

# Create study with pruner
study = optuna.create_study(
    direction='maximize',
    pruner= MedianPruner(n_startup_trials=5, n_warmup_steps=10),
    sampler=TPESampler(seed=randy)
)

# Run with callbacks
study.optimize(
    objective,
    n_trials=400, 
    callbacks=[print_best_callback, save_best_callback],
    n_jobs=-1,
    show_progress_bar=False
)

# After study.optimize() completes
best_params = study.best_params
print(f"Best parameters: {best_params}")
print(f"Best AUC: {study.best_value:.4f}")

[I 2025-07-07 10:27:55,106] A new study created in memory with name: no-name-97ab2362-56cf-448b-b95f-506c0a5a2603


[I 2025-07-07 10:28:11,360] Trial 0 finished with value: 0.6972604576481944 and parameters: {'n_estimators': 313, 'min_child_weight': 12, 'max_depth': 3, 'learning_rate': 0.7280849356408012, 'subsample': 0.6032474611442811, 'colsample_bytree': 0.34577546110928736, 'reg_lambda': 1.0564256819169864, 'reg_alpha': 0.00014743110884121556, 'scale_pos_weight': 3}. Best is trial 0 with value: 0.6972604576481944.


Trial 398: 0.7058 (best: 0.7087)
Best parameters: {'n_estimators': 415, 'min_child_weight': 10, 'max_depth': 9, 'learning_rate': 0.13511513493878266, 'subsample': 0.8074358236796195, 'colsample_bytree': 0.3361691464685002, 'reg_lambda': 3.2319036037247653, 'reg_alpha': 15.420312161747052, 'scale_pos_weight': 2}
Best AUC: 0.7087
CPU times: user 38min 35s, sys: 9min 48s, total: 48min 23s
Wall time: 7min 18s


In [18]:
xgb_final = Pipeline(steps=[
    ('preprocessor', preprocessor_xgb), 
    ('model', XGBClassifier(**best_params, eval_metric='logloss', random_state=randy))  
])

xgb_final.fit(X_train, y_train)

filename=f"../models/{token}_XGB_final.pkl"
with open(filename, "wb") as file:
    pickle.dump(xgb_final, file)
print(f"Model saved as {filename}")

Model saved as ../models/f11_XGB_final.pkl


## Final XGBoost Pipeline and Evaluation

**Pipeline Components:**
This final pipeline combines preprocessing and the optimized XGBoost model:
1. **Preprocessing**: Ordinal encoding for categoricals + passthrough for numerics (147 features preserved)
2. **XGBoost Model**: Gradient boosting with optimized hyperparameters across 9 key parameters for ensemble learning

**Model Persistence:**
The complete pipeline is saved as `{token}_XGB_final.pkl`, preserving both the fitted preprocessor and trained model for deployment.

**Standardized Evaluation:**
The `evaluate_and_save_pipeline()` function generates comprehensive metrics in the same format used across all models in this project, enabling direct performance comparison with linear models and supporting ensemble model development in subsequent notebooks.

In [19]:
# Open a trained model
# xgb_final = pd.read_pickle("../models/f04_XGB_final.pkl")

In [20]:
evaluate_and_save_pipeline(
    pipeline=xgb_final, 
    namestring='XGBoost',
    token=token, 
    X_train=X_train, 
    X_test=X_test, 
    y_train=y_train, 
    y_test=y_test)

{'model_version': 'f11_XGBoost',
 'accuracy': 0.6094924581142829,
 'precision': 0.5509637954335301,
 'recall': 0.8257115446114487,
 'f1_score': 0.6609215017064847,
 'specificity': np.float64(0.4246263215457528),
 'roc_auc': np.float64(0.7052126619520076),
 'y_test': array([1, 0, 0, ..., 1, 1, 0], dtype=int8),
 'y_train_pred': array([1, 0, 1, ..., 0, 1, 1]),
 'y_test_pred': array([1, 1, 1, ..., 1, 1, 1]),
 'y_test_pred_proba': array([0.62125766, 0.86658746, 0.8027118 , ..., 0.7682214 , 0.74429685,
        0.51019704], dtype=float32),
 'display_labels': array([0, 1]),
 'confusion_matrix': {'tn': np.int64(4659),
  'fp': np.int64(6313),
  'fn': np.int64(1635),
  'tp': np.int64(7746)},
 'roc_curve': {'fpr': array([0.        , 0.        , 0.        , ..., 0.97156398, 0.97156398,
         1.        ]),
  'tpr': array([0.00000000e+00, 1.06598444e-04, 1.17258288e-03, ...,
         9.99893402e-01, 1.00000000e+00, 1.00000000e+00]),
  'thresholds': array([       inf, 0.9870635 , 0.97624785, ..., 0